In [77]:
import pandas as pd
from random import randint

In [16]:
largest_cities = pd.read_html("https://www.infoplease.com/world/geography/major-cities-latitude-longitude-and-corresponding-time-zones")

In [21]:
df = largest_cities[0]

In [22]:
df.columns = ['City','lat1','lat2','log1','log2','time']

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    120 non-null    object
 1   lat1    120 non-null    int64 
 2   lat2    120 non-null    object
 3   log1    120 non-null    int64 
 4   log2    120 non-null    object
 5   time    120 non-null    object
dtypes: int64(2), object(4)
memory usage: 5.8+ KB


In [68]:
def city_row_to_coordinates(row):
  name = row[0]

  lat1 = int(row[1])
  lat2, lat_sign = row[2].split(' ')
  lat2 = int(lat2)
  lat_sign = 1 if lat_sign == 'N' else -1

  lon1 = int(row[3])
  lon2, lon_sign = row[4].split(' ')
  lon2 = int(lon2)
  lon_sign = 1 if lon_sign == 'W' else -1

  lat = lat_sign*(lat1 + lat2/60)
  lon = lon_sign*(lon1 + lon2/60)

  return name, lat, lon


def draw_coordinates(df):
  row = df.sample().values
  return city_row_to_coordinates(row[0])

In [69]:
print(draw_coordinates(df))
print(draw_coordinates(df))

('Port Moresby, Papua New Guinea', -9.416666666666666, -147.13333333333333)
('Darwin, Australia', -12.466666666666667, -130.85)


In [83]:
def prepare_random_link(df, size=200, alt=384400):  # 384 400km - average distance moon-earth
  name, lat, lon = draw_coordinates(df)
  year = randint(2000,2019)
  month = randint(1,12)
  day = randint(1,28)                               # not uniform for all days, but we dont need it to be uniform in days of month but in phases
  hour = randint(0,23)
  minute = randint(0,59)
  second = randint(0,59)

  link = f'http://www.fourmilab.ch/cgi-bin/Earth?imgsize={size}&opt=-l&lat={lat:.4f}&ns=North&lon={lon:.4f}&ew=West&alt={alt}&img=LRO_100m.evif&date=1&utc={year}-{month}-{day}+{hour}:{minute}:{second}'

  return link

In [84]:
print(prepare_random_link(df))
print(prepare_random_link(df))
print(prepare_random_link(df))

http://www.fourmilab.ch/cgi-bin/Earth?imgsize=200&opt=-l&lat=-42.8667&ns=North&lon=-147.3167&ew=West&alt=384400&img=LRO_100m.evif&date=1&utc=2002-1-19+15:57:17
http://www.fourmilab.ch/cgi-bin/Earth?imgsize=200&opt=-l&lat=47.5000&ns=North&lon=-19.0833&ew=West&alt=384400&img=LRO_100m.evif&date=1&utc=2014-6-8+17:31:13
http://www.fourmilab.ch/cgi-bin/Earth?imgsize=200&opt=-l&lat=52.3667&ns=North&lon=-4.8833&ew=West&alt=384400&img=LRO_100m.evif&date=1&utc=2014-3-17+16:27:12
